In [1]:
import pandas as pd
import itertools

In [2]:
prediction_dir = '../predictions/'
triplets_dir = '../data/triplets/'
drugbank_dir = '../data/drugbank/'

In [3]:
drug_name_map = pd.read_csv(drugbank_dir + 'drug_id_name_map.csv', sep=',', index_col=[0])
food_name_map = pd.read_csv(triplets_dir + 'food_name.tsv', sep='\t', index_col=[0])
food_compound_map = pd.read_csv(triplets_dir + 'compounds_names.tsv', sep='\t', index_col=[0])

In [4]:
drug_ids = drug_name_map.id
drug_names = drug_name_map.drug_name
drug_id_map_dict = dict(zip(drug_ids, drug_names))

food_ids = food_name_map.public_id
food_names = food_name_map.name
food_id_map_dict = dict(zip(food_ids, food_names))

compound_ids = food_compound_map.compound_id
compound_names = food_compound_map.name
compound_id_map_dict = dict(zip(compound_ids, compound_names))

In [5]:
common_drugs = pd.read_csv('../data/common_drugs.csv', sep=';')
common_drugs_ids = common_drugs.DrugBank_id.values
common_drugs_ids

array(['DB01558', 'DB08813', 'DB04817', 'DB00343', 'DB00829', 'DB00675',
       'DB00316', 'DB04817', 'DB00996', 'DB01009', 'DB01050', 'DB01373',
       'DB00451', 'DB00099'], dtype=object)

In [6]:
drug_id = common_drugs_ids[1]

predictions = pd.read_csv(prediction_dir + 'complex_' + drug_id + '_increase_adverse_effects_promising.csv', sep=',', index_col=[0])

# keep just drug/food/food compound predictions
predictions['node_type'] = list(itertools.repeat('xxx', predictions.shape[0]))
predictions.loc[predictions['tail_label'].str.contains("DB\d+", regex=True), 'node_type'] = "drug"
predictions.loc[predictions['tail_label'].str.contains("FDB"), 'node_type'] = "food_compound"
predictions.loc[predictions['tail_label'].str.contains("FOOD"), 'node_type'] = "food"
predictions = predictions.loc[predictions['node_type'] != 'xxx']
predictions

,tail_id,score,tail_label,in_validation,in_testing,node_type
3014,3014,26.459930,DB01109,False,False,drug
2349,2349,25.090981,DB00407,False,False,drug
2745,2745,21.839769,DB00823,False,False,drug
3709,3709,21.409637,DB08813,False,False,drug
4504,4504,21.326466,DB12768,False,False,drug
4182,4182,21.012867,DB11100,False,False,drug
2871,2871,20.433765,DB00957,False,False,drug
1990,1990,20.089470,DB00013,False,False,drug
3582,3582,19.950003,DB06605,False,False,drug
3155,3155,19.819180,DB01254,False,False,drug


In [7]:
# assign entity names to ids
for row in predictions.iterrows():
    tail = row[1].tail_label
    node_type = row[1].node_type
    
    if node_type == 'drug':
        tail_name = drug_id_map_dict[tail]
    elif node_type == 'food':
        tail_name = food_id_map_dict[tail]
    else:
        tail_name = compound_id_map_dict[tail]
        
    print(tail, tail_name)    

DB01109 Heparin
DB00407 Ardeparin
DB00823 Ethynodiol diacetate
DB08813 Nadroparin
DB12768 BCG vaccine
DB11100 Allantoin
DB00957 Norgestimate
DB00013 Urokinase
DB06605 Apixaban
DB01254 Dasatinib
DB00511 Acetyldigitoxin
DB00460 Verteporfin
DB05015 Belinostat
DB12016 Ponesimod
DB04812 Benoxaprofen
DB00959 Methylprednisolone
DB06209 Prasugrel
DB00974 Edetic acid
DB00078 Ibritumomab tiuxetan


In [12]:
# check if the predicted drug/food is in different interaction with the same drug in the training data

idx = drug_id

train_triplets = pd.read_csv(triplets_dir + 'train.tsv', sep='\t', index_col=[0])
filtered_triplets = train_triplets.loc[train_triplets.index == idx]
filtered_triplets.loc[filtered_triplets['tail'] == 'DB00823']

,relation,tail
head,,
